<a href="https://colab.research.google.com/github/semorrall/colab_uploads/blob/main/stephanie_sandbox/biol300_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!git clone https://github.com/semorrall/colab_uploads.git

Cloning into 'colab_uploads'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


# Problem 1: Comparing the 18S sequencing of three bird species (20 points)

a) Referencing the pre-class exercise we did for Friday, load in the `bird_18S.txt` file. 

b) Parse the information in this FASTA file, saving `chicken`, `duck`, and `turkey` variables as their corresponding DNA sequences.

c) Write a function with the following function call `SNP_counter(seq1, seq2)` that will return the number of difference (i.e. single nucleotide polymorphisms) between the two sequences inputted.

d) Using the function you wrote in part c, determine the number of difference between these birds' sequences. Display your results with the following print statements:

> `chicken and duck differ by __ basepairs`

> `chicken and turkey differ by __ basepairs`

> `duck and turkey differ by __ basepairs`

e) From your results, what can you conclude about the relative evolutionary relationship between these three birds? Does this make sense to you?


In [13]:

def call(a_file): #calls text and slices it at the start of each title
  if ".txt" not in a_file:
    a_file = path_lookup(a_file) 
  data = open(a_file).read() #find a way to permantly import the file
  return data

def path_lookup(nick_name): #set to only find files from the rapositiry but can be motified to pull from more locations
  name_dict = {'salmon': "salmonella_spi1_region.txt" , 'birds' :  "bird_18S.txt" , 'covid' : "6xlu.txt"}
  file_path = "/content/colab_uploads/" + name_dict[nick_name]
  return file_path


def format_type1(file): #takes output from call_seq and turns it into two lists: one of titles and one of sequences 
  data = call(file) 

  data = data.split('>')[1:] #turns data into list of strings starting with titles
   
  names = []
  seqs = []
  for i in data:
    i = i.split("\n", 1)
    names.append(i[0])
    seqs.append(i[1].replace("\n", ""))
   
  return names , seqs

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [14]:

def SNP_counter(seq1, seq2): #check this for bugs
  diff_count = 0
  for (i , b) in zip(list(seq1) , list(seq2)):
    if i != b:
      diff_count += 1

  return diff_count

def organize_inputs(a, b, names , seqs): #can completely remove if only accepting one input type
  if type(a) is str:
    ordered = [a , b , seqs[names.index(a)] , seqs[names.index(b)]] #sol = nameA, nameB , seqA , seqB
  else:
    ordered = [names[int(a)] , names[int(b)] , seqs[int(a)] , seqs[int(b)]] #could move the seq int to counter
  return ordered

def seq_compair(seq1, seq2, file_name): #alternative to if name equals main file #accepts sequence name or index
  
  names , seqs = format_type1(file_name) 
  ordered = organize_inputs(seq1, seq2, names , seqs)
  diff_count = SNP_counter(ordered[2], ordered[3])

  out = f'{ordered[0]} and {ordered[1]} differ by {diff_count} basepairs'
  return out



In [17]:
print(seq_compair("chicken", "turkey", "birds"))

chicken and turkey differ by 2 basepairs


# Problem 2: Using GC content to find the pathogenticy island in *Salmonella*

We've spent some time in this class computing the GC content of various sequences, but we will now apply that skill to address a question about *Salmonella*: where does it's pathogenicity arise from? GC content can vary across species, so regions of the *Salmonella* genome that differ greatly in GC content tend to suggest regions that arose from Horizontal Gene Transfer (HGT), meaning it picked up the gene from another organism. Such swapping of genes is very common in bacteria, and it is suspected that the genes that make *Salmonella* pathogenic arose from HGT, meaning those genes will have a different GC content from the rest of the genome. The following exercises will walk you through the process of uncovering this 'pathogenicity island' yourself!

## Problem 2.1 (10 points)

This time, we are interested not just of the GC content of an *entire* sequence, but of chunks along a genome, allowing us to find regions of lower or higher GC content. Write a function that divides a sequence into blocks and computes the GC content for each block, returning a list. The function signature should look like

`GC_blocks(seq, block_size)`

To be clear, if `seq = 'ATGACTACGT'` and `block_size = 4`, the blocks to be considered are

> `ATGA` and `CTAC`

and the function should return `[0.25, 0.5]`. Note that the blocks are non-overlapping and that we don't bother with the end of the sequence that does not fit completely in a block. **Note: Make sure to call the `GC_content` function from before when writing this new function!** I've included it here for you to use.

In [ ]:
def split_seq(seq, block_size): #slices a string into a deired number of characters
  codons = []
  for index in range(0, len(seq) - block_size + 1, block_size):
    codon = seq[index: index + block_size]
    codons.append(codon)
  return codons

def GC_blocks(seq, block_size): #takes in a string, splits into blocks and makes a list of the GC content of each block
  codons = split_seq(seq, block_size) #makes a list of codons

  GC_content = []
  for i in codons:
    G = i.count("G") 
    C = i.count("C")
    GC_ratio = ((G+C)/len(i))
    GC_content.append(GC_ratio)
  
  return GC_content 

In [ ]:
GC_blocks('ATGACTACGT', 4)

[0.25, 0.5]

## Problem 2.2 (15 points)

Write a function that takes as input a sequence, block size, and a threshold GC content, and returns a visualization of the sequence, where `'*'` designates a block that is below the threshold, and `'_'` designates a block that is above the threshold. You would call the function like this:

> `mapped_seq = GC_map(seq, block_size, gc_thresh)`

For example,

> `GC_map('ATGACTACGT', 4, 0.4)`

returns `'*_'`, indicating the first block has a GC content below 0.4, while the second block has a GC content above 0.4. Note that bases not included in GC blocks are truncated from the visualization.

In [ ]:
def gc_thresh_test(seq, block_size, gc_thresh):
  
  GC_content = GC_blocks(seq, block_size) #makes a list of codons GC content as a ratio

  thresh_results = ""
  for i in GC_content:
    if float(i) >= float(gc_thresh): 
      thresh_results += '_'
    else:
      thresh_results += '*'
  
  return thresh_results

In [ ]:
gc_thresh_test('ATGACTACGT', 4, 0.4)

'*_'

# Problem 2.3 (10 points)

Let's take your GC_map function for a spin, trying it on a portion of the Salmonella genome! First, load in the 'salmonella_spi1_region.txt' file and store the sequence as a single string with no gaps.

In [ ]:
def format_type2(file): #one tiltle and one seq
  data = call(file)  #import data                    
  seq = data.split("\n", 1)[-1] #issolate seq
  seq = seq.replace("\n", "") #remove line break
  return seq



...


## Problem 2.4 (5 points)

Run your `GC_map()` function on the salmonella sequence with `block_size = 1000` and  `gc_thresh = 0.45`. 

Looking at your visualization of GC content, about how far through the sequence (e.g. 10%, 80%) do you notice a region of lower GC content blocks? This is the pathogenicity island! This region is suspected to be where the genes that make *Salmonella* pathogenic lie!

In [ ]:
gc_thresh_test(format_type2('salmon'), 1000, .45) #find better alternative to a functin in a function

'_______________________________________****___***_*_**______**_____*_*___***_**_____________________________________*_____________*________________________*____________________________________________'

In [ ]:
a = "dsaefhiuwehfiquwe"

start = a.index("s")
end = a.index("f")

a = a[start : end]
print(a)

sae


# Problem 3: Exploring a PDB file (30 points)

For this last problem, we will explore a new type of data file, a protein database, or PDB file. We will be looking at SARS-COV2 spike protein [found here on PDB website](https://www.rcsb.org/structure/6XLU), although you will be manipualting the .txt file for this problem.

You may wish to open the file to get a sense of its structure. If you scroll down far enough, you will see a bunch of lines that begin with `ATOM`. These lines are the ones that describe the structure of the protein, by specifying the precise x-, y-, and z-coordinates of each atom in the structure. Each line contains information like atom number, atom identity (carbon, nitrogen, etc.), which amino acid the atom belongs to, and which subunit chain the amino acid belongs to. As an example, I show the first line below:
> `ATOM   1     N N   . ALA A  1 14   ? 137.193 160.456 208.534 1.00 62.75 ? 27   ALA A N   1`
 
See the [PDB formatting guidelines](http://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM) for a full description of what each column of information contains.

a) Read in the `'6xlu.txt'` file. 

b) Parse the information from the file to only get the ATOMs found in chain `'A'`. Note the chainID can be found in the 25th index of a given line in this file. Save each ATOM line corresponding to chain `'A'` to a list.

c) Display the first 10 and last 10 lines of your list to make sure you've isolated only the atoms we care about.

d) How many atoms are in chain `'A'`?

e) How many amino acids are in chain `'A'`?

f) Modify your code to repeat the process for chains `'B'` and `'C'`. How many amino acids do they have? What do you suspect this implies for the overall structure of the spike protein? 

Saving 6xlu.txt to 6xlu.txt


In [ ]:
def format_covid(name):
  seq = call(name)
  covid_prots = seq[seq.index("ATOM   1    ") : seq.index("\nHETATM 24873 C ")]
  covid_prots = covid_prots.split('\n')
  return covid_prots

In [ ]:
def find_chain(seqs , letter):

  chain = []
  for i in seqs:
    if i[25] == letter:
      chain.append(i)
  return chain



In [ ]:
def num_amino_acids(entries):
  acid_names = []
  for i in entries: #can this for loop be done in one line?
    acid_names.append(i[21: 24])
  num_aa = len(set(acid_names))
  return num_aa


def find_covid_chain_info(file_name , letter): #can this be made more modular? maybe make the output a second function that is called by the input covid?

  seq = format_covid(file_name)

  chain = find_chain(seq ,letter)

  num_atoms = len(chain)

  num_aa = num_amino_acids(chain)

  first_last = chain[0:10] + chain[-10:]

  out = f'''
There are {num_atoms} atoms and {num_aa} amino acids in the {letter} chain of the ATOMS portion of the data set.
Printed below are the first 10 and the last 10 lines from the chain {letter} data set:' #can I get that to print as a chart?
{first_last}'''

  return out

find_covid_chain_info('covid' , "A")


"\nThere are 8276 atoms and 20 amino acids in the A chain of the ATOMS portion of the data set.\nPrinted below are the first 10 and the last 10 lines from the chain A data set:' #can I get that to print as a chart?\n['ATOM   1     N N   . ALA A  1 14   ? 137.193 160.456 208.534 1.00 62.75 ? 27   ALA A N   1 ', 'ATOM   2     C CA  . ALA A  1 14   ? 136.846 161.310 207.402 1.00 62.75 ? 27   ALA A CA  1 ', 'ATOM   3     C C   . ALA A  1 14   ? 138.012 162.215 207.025 1.00 62.75 ? 27   ALA A C   1 ', 'ATOM   4     O O   . ALA A  1 14   ? 138.903 162.453 207.837 1.00 62.75 ? 27   ALA A O   1 ', 'ATOM   5     C CB  . ALA A  1 14   ? 135.613 162.138 207.720 1.00 62.75 ? 27   ALA A CB  1 ', 'ATOM   6     N N   . TYR A  1 15   ? 137.999 162.715 205.789 1.00 61.90 ? 28   TYR A N   1 ', 'ATOM   7     C CA  . TYR A  1 15   ? 139.055 163.573 205.274 1.00 61.90 ? 28   TYR A CA  1 ', 'ATOM   8     C C   . TYR A  1 15   ? 138.468 164.515 204.239 1.00 61.90 ? 28   TYR A C   1 ', 'ATOM   9     O O   . T

In [ ]:
find_covid_chain_info('covid' , "B")

"\nThere are 8302 atoms and 20 amino acids in the B chain of the ATOMS portion of the data set.\nPrinted below are the first 10 and the last 10 lines from the chain B data set:' #can I get that to print as a chart?\n['ATOM   8277  N N   . ALA B  1 14   ? 234.997 157.197 208.500 1.00 60.77 ? 27   ALA B N   1 ', 'ATOM   8278  C CA  . ALA B  1 14   ? 234.377 156.400 207.445 1.00 60.77 ? 27   ALA B CA  1 ', 'ATOM   8279  C C   . ALA B  1 14   ? 233.029 156.988 207.044 1.00 60.77 ? 27   ALA B C   1 ', 'ATOM   8280  O O   . ALA B  1 14   ? 232.388 157.677 207.835 1.00 60.77 ? 27   ALA B O   1 ', 'ATOM   8281  C CB  . ALA B  1 14   ? 234.218 154.959 207.892 1.00 60.77 ? 27   ALA B CB  1 ', 'ATOM   8282  N N   . TYR B  1 15   ? 232.602 156.702 205.815 1.00 61.88 ? 28   TYR B N   1 ', 'ATOM   8283  C CA  . TYR B  1 15   ? 231.383 157.272 205.261 1.00 61.88 ? 28   TYR B CA  1 ', 'ATOM   8284  C C   . TYR B  1 15   ? 230.860 156.355 204.172 1.00 61.88 ? 28   TYR B C   1 ', 'ATOM   8285  O O   . T

In [ ]:
find_covid_chain_info('covid' , "C")

"\nThere are 8294 atoms and 20 amino acids in the C chain of the ATOMS portion of the data set.\nPrinted below are the first 10 and the last 10 lines from the chain C data set:' #can I get that to print as a chart?\n['ATOM   16579 N N   . ALA C  1 14   ? 188.766 243.345 208.464 1.00 60.93 ? 27   ALA C N   1 ', 'ATOM   16580 C CA  . ALA C  1 14   ? 189.769 243.221 207.410 1.00 60.93 ? 27   ALA C CA  1 ', 'ATOM   16581 C C   . ALA C  1 14   ? 189.993 241.762 207.037 1.00 60.93 ? 27   ALA C C   1 ', 'ATOM   16582 O O   . ALA C  1 14   ? 189.798 240.871 207.861 1.00 60.93 ? 27   ALA C O   1 ', 'ATOM   16583 C CB  . ALA C  1 14   ? 191.073 243.861 207.845 1.00 60.93 ? 27   ALA C CB  1 ', 'ATOM   16584 N N   . TYR C  1 15   ? 190.401 241.522 205.790 1.00 62.77 ? 28   TYR C N   1 ', 'ATOM   16585 C CA  . TYR C  1 15   ? 190.644 240.176 205.294 1.00 62.77 ? 28   TYR C CA  1 ', 'ATOM   16586 C C   . TYR C  1 15   ? 191.760 240.219 204.265 1.00 62.77 ? 28   TYR C C   1 ', 'ATOM   16587 O O   . T

Part f)

Chain A , B, and C each have 20 amino acids. This implies that the overall structure of the spike protiens are bold text

# How long did this take? 

With a new course and new assignments, I want to be conscientious with how much time this course takes. Please let me know how long this took, so I can adjust future homeworks if needed.

# References

If you referenced any external sources for completing this homework, please list them below. (Just the links are fine.)

# Submitting your homework

Please make sure to state what each group member contribute and have each group member "sign off" that they agree they are satisfied with the final submission of this homework.

You will submit this homework (and all subsequent homeworks via GitHub). Unless you have an approved extension or opt to submit the homework late (with a 10% deduction per day), your homework will be graded based on what is submitted on GitHub at the time of the deadline. So don't forget to push! 